In [2]:
def date_posted(days) -> str:
    return "r" + str(days * 86400)

date_posted(7)

'r604800'

In [69]:
from pydantic import BaseModel,Field,computed_field
from typing import List
from enum import Enum

In [70]:
class JobType(str,Enum):
    REMOTE = 'remote'
    HYBRID = 'hybrid'
    ONSITE = 'onsite'

class ExperienceLevel(str, Enum):
    Internship = "Internship"
    Entry_level = "Entry_level"
    Associate = "Associate"
    Mid_Senior_level = "Mid_Senior_level"
    Director = "Director"
    Executive = "Executive"

In [ ]:
class JobInfo(BaseModel):
    title : str | None = Field(default=None,description= "Primary job title or role to search for.This represents the main occupation or position of interest." ,examples=['AI engineer','Data Scientist','SQL','Java','Software Engineer'])
    location : str | None = Field(default=None,description= "The name of the country where job needs to be find, If any city name is entered then think of the contry in which the city exist",examples=['India','America'])
    days : int | None = Field(default=7,description= "Job posted within the last days ",examples=[1,3,7,14])
    companyName : List[str] | None = Field(default=None,description= "The List of companies which needs to consider first or which is only needs to considered, ordered by priority ",examples=['Google','Microsoft'])
    companyId : List[str] | None = Field(default=None,description= "The List of Ids of the companies which needs to consider first or which is only needs to considered, ordered by priority ",examples=['21345','5567483'])
    skipJobId : List[str] | None = Field(default=None,description= "The List of Ids of the companies which needs to  be skiped or not considered",examples=['21345','5567483'])
    jobType : List[JobType] | None = Field(default=None,description= "This is the list of type of job the user preferred, ordered by priority ",examples=[['remote','hybrid'],['onsite']])
    experience_level : List[ExperienceLevel] | None = Field(default=None,description= "Preferred experience levels for the job, ordered by priority (from most to least preferred).")
    # typeOfContract : List[str] | None = Field(default=None,description= "The type of the ontract preffered by the user rankwise ",examples=[['Part_time','Full_time']])
    limit : int | None = Field(default=3,description= "The number of jobs the user wants to find even if the user will say a big number limit it up to 3",le=3,ge=1,examples=[3,1,2])
    
    @computed_field
    @property
    def datePosted(self) -> str:
        return "r" + str(self.days * 86400)

    @computed_field
    @property
    def remote(self) -> List[int] | None:
        if not self.jobType:
            return None

        mapping = {
            JobType.ONSITE: '1',
            JobType.REMOTE: '2',
            JobType.HYBRID: '3',
        }

        return [mapping[jt] for jt in self.jobType]
    
    @computed_field
    @property
    def experienceLevel(self) -> List[str] | None:
        if not self.experience_level:
            return None

        mapping = {
            ExperienceLevel.Internship: "1",
            ExperienceLevel.Entry_level: "2",
            ExperienceLevel.Associate: "3",
            ExperienceLevel.Mid_Senior_level: "4",
            ExperienceLevel.Director: "5",
            ExperienceLevel.Executive: "6",
        }

        return [mapping[ctype] for ctype in self.experience_level]  
    
    

In [ ]:
""""1"	Internship
"2"	Entry level
"3"	Associate
"4"	Mid-Senior level
"5"	Director
"6"	Executive"""

In [80]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite')

In [81]:
user_prompt = "I want to find the jobs for AI engineering remote internship find 100 jobs in Ahmedabad"

In [82]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(template="""
        You are job searching helping agent you need to pass the 
        data to job searchng API for that you need to structure the 
        output from the given user prompt : {user_prompt}
        """,
        input_variables=['user_prompt'])

In [83]:
job_info_agent = (
    prompt | 
    llm.with_structured_output(JobInfo)
    )

In [84]:
res = job_info_agent.invoke({'user_prompt':user_prompt})

c:\Users\DELL\miniconda3\envs\MCP\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [field_name='remote', input_value='2', input_type=str])
  return self.__pydantic_serializer__.to_python(


In [85]:
res

JobInfo(title='AI engineer', location='Ahmedabad', days=7, companyName=None, companyId=None, skipJobId=None, jobType=[<JobType.REMOTE: 'remote'>], experience_level=[<ExperienceLevel.Internship: 'Internship'>], limit=3, datePosted='r604800', remote=['2'], experienceLevel=['1'])

In [16]:
type(res)

__main__.JobInfo

In [86]:
job_info_dict = res.model_dump()

In [87]:
job_info_dict

{'title': 'AI engineer',
 'location': 'Ahmedabad',
 'days': 7,
 'companyName': None,
 'companyId': None,
 'skipJobId': None,
 'jobType': [<JobType.REMOTE: 'remote'>],
 'experience_level': [<ExperienceLevel.Internship: 'Internship'>],
 'limit': 3,
 'datePosted': 'r604800',
 'remote': ['2'],
 'experienceLevel': ['1']}

In [19]:
class Job_Info_state(BaseModel):
    title : str  | None
    location : str | None
    datePosted : str | None
    companyName : List[str] | None
    companyId : List[str] | None
    skipJobId : List[str] | None
    remote : List[str] | None
    experienceLevel : List[str] | None
    contractType : List[str] | None
    limit : int | None
    
{
  "title": "Java",
  "location": "New York",
  "datePosted": "r604800",
  "companyName": [
    "Google",
    "Apple"
  ],
  "companyId": [
    "1441",
    "162479"
  ],
  "contractType": [
    "F",
    "P"
  ],
  "experienceLevel": [
    "1",
    "2"
  ],
  "remote": [
    "2",
    "3"
  ],
  "limit": 100,
  "urlParam": [
    {
      "key": "f_TPR",
      "value": "r3600"
    }
  ],
  "skipJobId": [
    "4219847745",
    "4219847746"
  ]
}

{'title': 'Java',
 'location': 'New York',
 'datePosted': 'r604800',
 'companyName': ['Google', 'Apple'],
 'companyId': ['1441', '162479'],
 'contractType': ['F', 'P'],
 'experienceLevel': ['1', '2'],
 'remote': ['2', '3'],
 'limit': 100,
 'urlParam': [{'key': 'f_TPR', 'value': 'r3600'}],
 'skipJobId': ['4219847745', '4219847746']}

In [20]:
job_info_obj = Job_Info_state.model_validate(job_info_dict)

In [21]:
job_info_obj

Job_Info_state(title='AI engineer', location='Ahmedabad', datePosted='r604800', companyName=None, companyId=None, skipJobId=None, remote=['2'], experienceLevel=None, contractType=['I'], limit=3)

In [22]:
Job_Info_state(**job_info_dict)

Job_Info_state(title='AI engineer', location='Ahmedabad', datePosted='r604800', companyName=None, companyId=None, skipJobId=None, remote=['2'], experienceLevel=None, contractType=['I'], limit=3)

In [88]:
job_info_dict

{'title': 'AI engineer',
 'location': 'Ahmedabad',
 'days': 7,
 'companyName': None,
 'companyId': None,
 'skipJobId': None,
 'jobType': [<JobType.REMOTE: 'remote'>],
 'experience_level': [<ExperienceLevel.Internship: 'Internship'>],
 'limit': 3,
 'datePosted': 'r604800',
 'remote': ['2'],
 'experienceLevel': ['1']}

In [89]:
clean_payload = res.model_dump(
    mode="json",
    exclude = ['days','jobType','experience_level'],
    exclude_none=True
)

c:\Users\DELL\miniconda3\envs\MCP\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [field_name='remote', input_value='2', input_type=str])
  return self.__pydantic_serializer__.to_python(


In [90]:
clean_payload

{'title': 'AI engineer',
 'location': 'Ahmedabad',
 'limit': 3,
 'datePosted': 'r604800',
 'remote': ['2'],
 'experienceLevel': ['1']}

In [91]:
load_dotenv()

True

In [ ]:
import os 
APIFY_TOKEN = os.getenv('APIFY_TOKEN') 
APIFY_ACTOR_NAME = os.getenv('APIFY_ACTOR_NAME')

In [30]:
APIFY_ACTOR_NAME

'valig/linkedin-jobs-scraper'

In [ ]:
from apify_client import ApifyClient

apify_client = ApifyClient(APIFY_TOKEN)

# Define the input for the Actor
actor_input = {'title': 'AI engineer',
 'location': 'India',
 'limit': 3,
 'datePosted': 'r604800',
 'remote': ['2'],
 'experienceLevel': ['1']}
# actor_input = clean_payload

# Run an Actor with an input
print("Running the Actor...")
actor_name = APIFY_ACTOR_NAME
actor_run = apify_client.actor(actor_name).start(run_input=actor_input)

print("🚀 Actor was started")
print("💾 Check your run here: https://console.apify.com/actors/runs/%(id)s" % {"id": actor_run["id"]})

Running the Actor...
🚀 Actor was started
💾 Check your run here: https://console.apify.com/actors/runs/deA7vtUpFTc1nNric


In [100]:
class Job(BaseModel):
    id : str
    url : str
    title : str
    location: str
    companyName: str 
    companyUrl:str
    recruiterName:str
    recruiterUrl:str
    experienceLevel: str
    contractType:str
    workType:str
    sector: str
    salary:  str
    applyType: str
    applyUrl:str 
    postedTimeAgo:str  
    postedDate: str
    applicationsCount: str
    description:str

In [102]:
l = []
for item in apify_client.dataset(actor_run["defaultDatasetId"]).iterate_items():
    print(item)
    print(type(item))
    l.append(Job(**item))
print(l)

{'id': '4347051914', 'url': 'https://www.linkedin.com/jobs/view/machine-learning-engineer-intern-at-upstart-4347051914', 'title': 'Machine Learning Engineer Intern', 'location': 'United States', 'companyName': 'Upstart', 'companyUrl': 'https://www.linkedin.com/company/upstart-network', 'recruiterName': '', 'recruiterUrl': '', 'experienceLevel': 'Internship', 'contractType': 'Internship', 'workType': 'Engineering and Information Technology', 'sector': 'Financial Services', 'salary': '$141,000.00/yr - $150,000.00/yr', 'applyType': 'EXTERNAL', 'applyUrl': '', 'postedTimeAgo': '3 days ago', 'postedDate': '2025-12-13T00:00:00.000Z', 'applicationsCount': 'Over 200 applicants', 'description': "About UpstartUpstart is the leading AI lending marketplace partnering with banks and credit unions to expand access to affordable credit. By leveraging Upstart's AI marketplace, Upstart-powered banks and credit unions can have higher approval rates and lower loss rates across races, ages, and genders, w

In [103]:
l

[Job(id='4347051914', url='https://www.linkedin.com/jobs/view/machine-learning-engineer-intern-at-upstart-4347051914', title='Machine Learning Engineer Intern', location='United States', companyName='Upstart', companyUrl='https://www.linkedin.com/company/upstart-network', recruiterName='', recruiterUrl='', experienceLevel='Internship', contractType='Internship', workType='Engineering and Information Technology', sector='Financial Services', salary='$141,000.00/yr - $150,000.00/yr', applyType='EXTERNAL', applyUrl='', postedTimeAgo='3 days ago', postedDate='2025-12-13T00:00:00.000Z', applicationsCount='Over 200 applicants', description="About UpstartUpstart is the leading AI lending marketplace partnering with banks and credit unions to expand access to affordable credit. By leveraging Upstart's AI marketplace, Upstart-powered banks and credit unions can have higher approval rates and lower loss rates across races, ages, and genders, while simultaneously delivering the exceptional digita

In [107]:
for job in l:
    print(job.description)

About UpstartUpstart is the leading AI lending marketplace partnering with banks and credit unions to expand access to affordable credit. By leveraging Upstart's AI marketplace, Upstart-powered banks and credit unions can have higher approval rates and lower loss rates across races, ages, and genders, while simultaneously delivering the exceptional digital-first lending experience their customers demand. More than 80% of borrowers are approved instantly, with zero documentation to upload.Upstart is a digital-first company, which means that most Upstarters live and work anywhere in the United States. However, we also have offices in San Mateo, California; Columbus, Ohio; Austin, Texas; and New York City, NY (opening Summer 2026).Most Upstarters join us because they connect with our mission of enabling access to effortless credit based on true risk. If you are energized by the impact you can make at Upstart, we’d love to hear from you!The TeamMachine Learning is at the heart of Upstart’s

In [34]:
actor_input

{'title': 'AI Engineer',
 'location': 'India',
 'datePosted': 'r604800',
 'experienceLevel': ['1', '2'],
 'remote': ['2', '3'],
 'limit': 1}

In [35]:
clean_payload

{'title': 'AI engineer',
 'location': 'Ahmedabad',
 'days': 7,
 'jobType': ['remote'],
 'typeOfContract': ['Internship'],
 'limit': 3,
 'datePosted': 'r604800',
 'remote': ['2'],
 'contractType': ['I']}

In [ ]:
clean_payload.pop('days')
clean_payload.pop('jobType')
clean_payload.pop('experience_level')

['Internship']

In [39]:
clean_payload

{'title': 'AI engineer',
 'location': 'Ahmedabad',
 'limit': 3,
 'datePosted': 'r604800',
 'remote': ['2'],
 'contractType': ['I']}

In [ ]:
actor_input = {
  "title": "AI Engineer",
  "location": "Ahmedabad",
  "limit": 3,
  "datePosted": "r604800",
  "remote": ["2"]
}

In [55]:
actor_input

{'title': 'AI Engineer',
 'location': 'Ahmedabad',
 'limit': 1,
 'datePosted': 'r604800',
 'remote': ['2'],
 'contractType': ['F']}

In [59]:
clean_payload_final.pop('contractType')

['I']

In [62]:
clean_payload_final

{'title': 'AI engineer',
 'location': 'Ahmedabad',
 'limit': 3,
 'datePosted': 'r604800',
 'remote': ['2']}

In [58]:
from apify_client import ApifyClient

apify_client = ApifyClient(APIFY_TOKEN)

# Define the input for the Actor
# actor_input = {
#   "title": "AI Engineer",
#   "location": "India",
#   "datePosted": "r604800",
#   "experienceLevel": [
#     "1",
#     "2"
#   ],
#   "remote": [
#     "2",
#     "3"
#   ],
#   "limit": 1,
# }

# Run an Actor with an input
print("Running the Actor...")
actor_name = APIFY_ACTOR_NAME
actor_run = apify_client.actor(actor_name).start(run_input=actor_input)

print("🚀 Actor was started")
print("💾 Check your run here: https://console.apify.com/actors/runs/%(id)s" % {"id": actor_run["id"]})

Running the Actor...
🚀 Actor was started
💾 Check your run here: https://console.apify.com/actors/runs/EQbZFXPvedQcWQLW4


In [65]:
from apify_client import ApifyClient

apify_client = ApifyClient(APIFY_TOKEN)

# Define the input for the Actor
actor_input = {
  "title": "AI Engineer",
  "location": "India",
  "datePosted": "r604800",
  "experienceLevel": [
    "1",
    "2"
  ],
  "remote": [
    "2",
    "3"
  ],
  "limit": 1,
  "experienceLevel" : ['0']
}

# Run an Actor with an input
print("Running the Actor...")
actor_name = APIFY_ACTOR_NAME
actor_run = apify_client.actor(actor_name).start(run_input=actor_input)

print("🚀 Actor was started")
print("💾 Check your run here: https://console.apify.com/actors/runs/%(id)s" % {"id": actor_run["id"]})

Running the Actor...


ApifyApiError: Input is not valid: Field input.experienceLevel.0 must be equal to one of the allowed values: "1", "2", "3", "4", "5", "6"